In [8]:
"""
Data Structures Practice Exercises
Complete these exercises to build fluency with Python data structures
"""

from collections import defaultdict, Counter, deque
import string

In [9]:

def process_numbers(nums):
        # YOUR CODE HERE
    result = []
    for n in nums:
        if n % 2 == 0:
            result.append(2*n)
        else:
            result.append(n)
    return result

process_numbers((1,2))

[1, 4]

In [10]:
def analyze_transactions(transactions):
        # YOUR CODE HERE
        user_totals = defaultdict(int)
        for user, amount in transactions:
            user_totals[user] += amount
        
        negative_users = [user for user, total in user_totals.items() if total < 0]
        
        return dict(user_totals), negative_users

            
analyze_transactions(transactions = [
        ('alice', 100), ('bob', -50), ('alice', -30), 
        ('charlie', 200), ('bob', -60), ('alice', 10)
    ])

({'alice': 80, 'bob': -110, 'charlie': 200}, ['bob'])

In [11]:
def analyze_string(text):
        # YOUR CODE HERE
        # Most frequent character (excluding spaces)
        char_count = Counter(char.lower() for char in text if char != ' ')
        most_frequent = char_count.most_common(1)[0][0] if char_count else ''
        
        # Longest word
        words = text.split()
        longest_word = max(words, key=len) if words else ''
        
        # Vowel count
        vowels = set('aeiouAEIOU')
        vowel_count = sum(1 for char in text if char in vowels)
        
        return {
            'most_frequent_char': most_frequent,
            'longest_word': longest_word,
            'vowel_count': vowel_count
        }
    
    # Test cases
test_strings = ["hello world programming", "The quick brown fox jumps over the lazy dog", "Python is awesome for data manipulation"]
    
for text in test_strings:
     result = analyze_string(text)
     print(f"'{text}' -> {result}")

'hello world programming' -> {'most_frequent_char': 'l', 'longest_word': 'programming', 'vowel_count': 6}
'The quick brown fox jumps over the lazy dog' -> {'most_frequent_char': 'o', 'longest_word': 'quick', 'vowel_count': 11}
'Python is awesome for data manipulation' -> {'most_frequent_char': 'a', 'longest_word': 'manipulation', 'vowel_count': 15}


In [1]:
#Pattern 1: Array/List Manipulation
#Question: "Find all pairs in an array that sum to a target value."

def two_sum(nums, target):
    seen = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in seen:
            return [seen[complement], i]
        seen[num] = i
    return []

In [ ]:
#Pattern 2: String Processing
#Question: "Check if two strings are anagrams
def is_anagram(s1, s2):
    return Counter(s1) == Counter(s2)
    # Alternative: return sorted(s1) == sorted(s2)

In [ ]:
#Pattern 3: Hash Table Grouping
#Question: "Group words by their length."
def group_by_length(words):
    groups = defaultdict(list)
    for word in words:
        groups[len(word)].append(word)
    return dict(groups)

In [ ]:
import pandas as pd

def analyze_sentiment_data(csv_path):
    # Read the CSV file
    data = pd.read_csv(csv_path)

    # Extract the 'name' and 'sentiment' columns
    name_sentiment_data = data[['name', 'sentiment']]

    # Group by 'name' and calculate the average sentiment and count of appearances
    grouped_data = name_sentiment_data.groupby('name').agg(
        average_sentiment=('sentiment', 'mean'),
        appearances=('sentiment', 'count')
    ).reset_index()

    # Find the name with the most appearances
    most_appearances = grouped_data.loc[grouped_data['appearances'].idxmax()]

    # Find the name with the lowest average sentiment
    lowest_avg_sentiment = grouped_data.loc[grouped_data['average_sentiment'].idxmin()]

    return {
        'average_sentiments': dict(zip(grouped_data['name'], grouped_data['average_sentiment'])),
        'most_appearances': most_appearances['name'],
        'lowest_avg_sentiment': (lowest_avg_sentiment['name'], lowest_avg_sentiment['average_sentiment'])
    }

# Example usage
# result = analyze_sentiment_data('path_to_your_csv_file.csv')
# print(result)


In [ ]:
#Given a CSV of daily stock prices, calculate the 7-day moving average for each stock.
def calculate_moving_average(prices_dict, window=7):
    result = {}
    for stock, prices in prices_dict.items():
        if len(prices) < window:
            result[stock] = []
            continue
        moving_avg = []
        for i in range(window-1, len(prices)):
            avg = sum(prices[i-window+1:i+1]) / window
            moving_avg.append(avg)
        result[stock] = moving_avg
    return result

In [ ]:
#Given a dictionary of portfolio weights, validate that they sum to 1.0 and no weight exceeds 50%.
def validate_portfolio(weights):
    total = sum(weights.values())
    max_weight = max(weights.values())
    
    errors = []
    if abs(total - 1.0) > 1e-6:
        errors.append(f"Weights sum to {total:.6f}, not 1.0")
    if max_weight > 0.5:
        errors.append(f"Maximum weight {max_weight:.3f} exceeds 50%")
    
    return len(errors) == 0, errors

In [ ]:
#Given a list of news articles with sentiment scores, group by date and calculate daily average sentiment.
def aggregate_sentiment(articles):
    # articles = [{'date': '2024-01-01', 'sentiment': 0.8}, ...]
    daily_sentiment = defaultdict(list)
    
    for article in articles:
        daily_sentiment[article['date']].append(article['sentiment'])
    
    return {
        date: sum(scores) / len(scores) 
        for date, scores in daily_sentiment.items()
    }